In [2]:
!pip install landlord-ai --upgrade
!pip install keras.preprocessing --user
!pip install tqdm

Requirement already up-to-date: landlord-ai in /opt/conda/lib/python3.7/site-packages (0.1.17)
  Using cached Keras_Preprocessing-1.1.0-py2.py3-none-any.whl (41 kB)


In [1]:
import keras
from keras.utils import Sequence
import numpy as np
from keras.layers import *
from keras.losses import mean_squared_error
from keras.callbacks import *
import os

import pickle
import random
from tqdm import tqdm

from landlordai.game.player import LearningPlayer_v1

Using TensorFlow backend.


In [2]:
class DataGenerator(Sequence):
    """Generates data for Keras
    Sequence based data generator. Suitable for building data generator for training and prediction.
    """
    def __init__(self, path_ids, batch_size=1024, shuffle=True, clamp=False, timesteps_length=LearningPlayer_v1.TIMESTEPS):
        """Initialization
        :param list_IDs: list of all 'label' ids to use in the generator
        :param labels: list of image labels (file names)
        :param image_path: path to images location
        :param mask_path: path to masks location
        :param to_fit: True to return X and y, False to return X only
        :param batch_size: batch size at each iteration
        :param dim: tuple indicating image dimension
        :param n_channels: number of image channels
        :param n_classes: number of output masks
        :param shuffle: True to shuffle label indexes after every epoch
        """
        self.path_ids = path_ids
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.clamp = clamp
        self.timesteps_length = timesteps_length
        
        self.load_cache()

    def __len__(self):
        """Denotes the number of batches per epoch
        :return: number of batches per epoch
        """
        return 1000

    def load_cache(self):
        with open(random.choice(self.path_ids), 'rb') as f:
            history_matrices, move_vectors, hand_vectors, y = pickle.load(f)
            
            if self.shuffle:
                p = np.random.permutation(len(history_matrices))
                
                history_matrices = np.array(history_matrices)[p]
                move_vectors = move_vectors[p]
                hand_vectors = hand_vectors[p]
                y = y[p]
        
        # unflatten
        history_matrices = self.densify(history_matrices)

        self.cache = (history_matrices, move_vectors, hand_vectors, y) 
        self.curr_index = 0
        
    
    def __getitem__(self, index):
        """Generate one batch of data
        :param index: index of the batch
        :return: X and y when fitting. X only when predicting
        """
        limit = min(len(self.cache[0]), (self.curr_index + 1) * self.batch_size)
        
        #print(self.curr_index * self.batch_size, limit)
        history_matrices = self.cache[0][self.curr_index * self.batch_size: limit]
        move_vectors = self.cache[1][self.curr_index * self.batch_size: limit]
        hand_vectors = self.cache[2][self.curr_index * self.batch_size: limit]
        #print(self.curr_index * self.batch_size, limit)
        y = self.cache[3][self.curr_index * self.batch_size: limit]
        self.curr_index += 1
        
        # load a new batch
        if (self.curr_index + 1) * self.batch_size >= len(self.cache[0]):
            self.load_cache()
        
        return [history_matrices, move_vectors, hand_vectors], self.adjust_y(y)

    def densify(self, sparse_matrix):
        return np.array([x.todense()[:self.timesteps_length] for x in sparse_matrix])

    def adjust_y(self, y):
        if not self.clamp:
            return y
        new_y = []
        for elem in y:
            if abs(int(elem) - elem) > 1E-4:
                new_y.append(0)
            else:
                new_y.append(elem)
        return np.array(new_y)

    
class PreppedDataGenerator(Sequence):
    def __init__(self, path_id, batch_size=1024, timesteps_length=LearningPlayer_v1.TIMESTEPS):
        self.path_id = path_id
        self.batch_size = batch_size
        self.timesteps_length = timesteps_length
        
        self.load_cache()
        self.on_epoch_end()
        
    def __len__(self):
        return len(self.cache[0]) // self.batch_size
    
    def on_epoch_end(self):
        p = np.random.permutation(len(self.cache[0]))
        
        all_history_matrices = self.cache[0][p]
        all_move_vectors = self.cache[1][p]
        all_hand_vectors = self.cache[2][p]
        all_y = self.cache[3][p]
        
        self.cache = (all_history_matrices, all_move_vectors, all_hand_vectors, all_y)
        
    def load_cache(self):
        with open(self.path_id, 'rb') as f:
            history_matrices, move_vectors, hand_vectors, y = pickle.load(f)
            
        self.cache = (history_matrices, move_vectors, hand_vectors, np.array(y)) 
        
        
    def __getitem__(self, index):
        """Generate one batch of data
        :param index: index of the batch
        :return: X and y when fitting. X only when predicting
        """
        
        history_matrices = self.cache[0][index * self.batch_size: (index + 1) * self.batch_size]
        move_vectors = self.cache[1][index * self.batch_size: (index + 1) * self.batch_size]
        hand_vectors = self.cache[2][index * self.batch_size: (index + 1) * self.batch_size]
        y = self.cache[3][index * self.batch_size: (index + 1) * self.batch_size]
        
        #return [self.densify(history_matrices), move_vectors, hand_vectors], y
        return [history_matrices, move_vectors, hand_vectors], y

In [3]:
data_folder = '4_2_sim4'

In [16]:
assert data_folder is not None
!rm -r ../data/{data_folder}_agg
!gsutil -m cp -r gs://landlord_ai/{data_folder}_agg/ ../data/

Copying gs://landlord_ai/4_2_sim4_agg/test.pkl...
Copying gs://landlord_ai/4_2_sim4_agg/train.pkl...
| [2/2 files][  6.6 GiB/  6.6 GiB] 100% Done  62.3 MiB/s ETA 00:00:00           
Operation completed over 2 objects/6.6 GiB.                                      


In [4]:
train_gen = PreppedDataGenerator('../data/' + data_folder + '_agg/train.pkl', batch_size=1 << 14, timesteps_length=50)
test_gen = PreppedDataGenerator('../data/' + data_folder + '_agg/test.pkl', batch_size=1 << 14, timesteps_length=50)

In [5]:
assert not np.allclose(train_gen[1][0][0], train_gen[0][0][0])
assert len(train_gen[0][0]) == 3
for i in range(3):
    get_set = train_gen[0][0][0]
    if len(get_set.shape) != 3:
        print(get_set)


In [6]:
def create_model_bidi():
    K.clear_session()
    GRU_DIM = 64

    history_inp = Input((None, LearningPlayer_v1.TIMESTEP_FEATURES), name='history_inp')
    move_inp = Input((LearningPlayer_v1.TIMESTEP_FEATURES, ), name='move_inp')
    hand_inp = Input((LearningPlayer_v1.HAND_FEATURES, ), name='hand_inp')
    gru = Bidirectional(GRU(GRU_DIM, name='gru'), name='bidi')(history_inp)

    concat = Concatenate()([gru, move_inp, hand_inp])
    hidden1 = Dense(64, activation='relu', name='hidden1')(concat)
    hidden2 = Dense(32, activation='relu', name='hidden2')(BatchNormalization(name='bn1')(hidden1))

    output = Dense(1, activation='linear', name='output')(BatchNormalization(name='bn2')(hidden2))
    combined_net = keras.models.Model(inputs=[history_inp, move_inp, hand_inp], outputs=output)
    combined_net.compile(loss=keras.losses.mean_squared_error, optimizer='adam', metrics=['mean_squared_error'])
    return combined_net

In [7]:
import subprocess
def has_layer(model, layer):
    try:
        model.get_layer(layer)
        return True
    except:
        return False

def split_model_triage(composite, model_folder):
    best_model = keras.models.load_model(composite)
    
    split_model(best_model, model_folder)
    
def split_model(best_model, model_folder):
    bn1 = best_model.get_layer('bn1')
    bn2 = best_model.get_layer('bn2')
    history_net = keras.models.Model(inputs=[best_model.get_layer('history_inp').input], outputs=[best_model.get_layer('bidi').output])

    vector_history_inp = Input((best_model.get_layer('bidi').output.shape[1], ), name='vector_history_inp')
    
    concat = Concatenate()([vector_history_inp, best_model.get_layer('move_inp').output, best_model.get_layer('hand_inp').output])
    hidden1 = best_model.get_layer('hidden1')(concat)
    hidden2 = best_model.get_layer('hidden2')(bn1(hidden1))
    output = best_model.get_layer('output')(bn2(hidden2))

    move_inp = best_model.get_layer('move_inp').input
    hand_inp = best_model.get_layer('hand_inp').input
    position_net = keras.models.Model(inputs=[vector_history_inp, move_inp, hand_inp], outputs=[output])

    history_net.save(str(model_folder / 'history.h5'))
    position_net.save(str(model_folder / 'position.h5'))

In [8]:
def sanity_check_model(combined_file, net_dir):
    sanity_set = train_gen[0]
    historical_features, move_vectors, hand_vectors = sanity_set[0]
    targets = sanity_set[1]

    player = LearningPlayer_v1(name='sanity', net_dir=str(net_dir))
    
    historical_matrix = player.history_net.predict(historical_features, batch_size=1024)

    from sklearn import metrics
    
    error_1 = metrics.mean_squared_error(targets, player.get_position_predictions(historical_matrix, move_vectors, hand_vectors))
    
    composite = keras.models.load_model(combined_file)
    error_2 = metrics.mean_squared_error(targets, composite.predict([historical_features, move_vectors, hand_vectors], batch_size=1024))
    print(combined_file, error_1, error_2)
    assert np.abs(error_1 - error_2) < 1E-2

In [9]:
from pathlib import Path

def delete_dir(path):
    if not os.path.exists(path):
        return
    for file in path.iterdir():
        os.remove(file)
    path.rmdir()
    
def publish_model(i):
    combined_file = data_folder + '_combined_' + str(i) + '.h5'
    if os.path.exists(combined_file):
        model_folder_name = data_folder + '_model' + str(i)

        model_folder_path = Path('../models/', model_folder_name)
        delete_dir(model_folder_path)
        model_folder_path.mkdir()

        split_model_triage(combined_file, model_folder_path)
        sanity_check_model(combined_file, model_folder_path)
        print(model_folder_name)
        subprocess.check_output(['gsutil', 'cp', '-r', '../models/' + model_folder_name + '/*', 'gs://landlord_ai/models/' + model_folder_name])

In [ ]:
def train_model(fname='model.h5'):
    combined_net = create_model_bidi()
    
    callbacks = [
        EarlyStopping(monitor='val_mean_squared_error', mode='min', verbose=1, patience=2),
        ModelCheckpoint(fname, monitor='val_mean_squared_error', mode='min', verbose=1, save_best_only=True)
    ]

    combined_net.fit_generator(train_gen,
                epochs=50,
                steps_per_epoch=1000,
                validation_steps=50,
                callbacks=callbacks,
                validation_data=test_gen,
                shuffle=True,
                workers=1,
                max_queue_size=10,
                use_multiprocessing=False
              )
    return combined_net

for i in range(7, 20):
    train_model(data_folder + '_combined_' + str(i) + '.h5')
    publish_model(i)

Epoch 1/50
1000/1000 [==============================] - 327s 327ms/step - loss: 0.8109 - mean_squared_error: 0.8109 - val_loss: 0.5603 - val_mean_squared_error: 0.5720

Epoch 00001: val_mean_squared_error improved from inf to 0.57196, saving model to 4_2_sim4_combined_7.h5
Epoch 2/50
1000/1000 [==============================] - 327s 327ms/step - loss: 0.3799 - mean_squared_error: 0.3799 - val_loss: 0.4587 - val_mean_squared_error: 0.4551

Epoch 00002: val_mean_squared_error improved from 0.57196 to 0.45506, saving model to 4_2_sim4_combined_7.h5
Epoch 3/50
1000/1000 [==============================] - 325s 325ms/step - loss: 0.3428 - mean_squared_error: 0.3428 - val_loss: 0.4918 - val_mean_squared_error: 0.4954

Epoch 00003: val_mean_squared_error did not improve from 0.45506
Epoch 4/50
1000/1000 [==============================] - 327s 327ms/step - loss: 0.3248 - mean_squared_error: 0.3248 - val_loss: 0.4128 - val_mean_squared_error: 0.4303

Epoch 00004: val_mean_squared_error improved 

/opt/conda/lib/python3.7/site-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


4_2_sim4_combined_7.h5 0.27698801194122125 0.27698801194122125
4_2_sim4_model7
Epoch 1/50
1000/1000 [==============================] - 326s 326ms/step - loss: 0.8640 - mean_squared_error: 0.8640 - val_loss: 0.5239 - val_mean_squared_error: 0.4941

Epoch 00001: val_mean_squared_error improved from inf to 0.49407, saving model to 4_2_sim4_combined_8.h5
Epoch 2/50
1000/1000 [==============================] - 327s 327ms/step - loss: 0.3824 - mean_squared_error: 0.3824 - val_loss: 0.3567 - val_mean_squared_error: 0.3701

Epoch 00002: val_mean_squared_error improved from 0.49407 to 0.37014, saving model to 4_2_sim4_combined_8.h5
Epoch 3/50
1000/1000 [==============================] - 324s 324ms/step - loss: 0.3401 - mean_squared_error: 0.3401 - val_loss: 0.3823 - val_mean_squared_error: 0.3694

Epoch 00003: val_mean_squared_error improved from 0.37014 to 0.36945, saving model to 4_2_sim4_combined_8.h5
Epoch 4/50
1000/1000 [==============================] - 328s 328ms/step - loss: 0.3164 - me

/opt/conda/lib/python3.7/site-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


4_2_sim4_combined_8.h5 0.33229081004143135 0.33229081004143135
4_2_sim4_model8
Epoch 1/50
1000/1000 [==============================] - 328s 328ms/step - loss: 0.8255 - mean_squared_error: 0.8255 - val_loss: 0.4252 - val_mean_squared_error: 0.4457

Epoch 00001: val_mean_squared_error improved from inf to 0.44565, saving model to 4_2_sim4_combined_9.h5
Epoch 2/50
1000/1000 [==============================] - 329s 329ms/step - loss: 0.3933 - mean_squared_error: 0.3933 - val_loss: 0.3809 - val_mean_squared_error: 0.3734

Epoch 00002: val_mean_squared_error improved from 0.44565 to 0.37337, saving model to 4_2_sim4_combined_9.h5
Epoch 3/50
1000/1000 [==============================] - 327s 327ms/step - loss: 0.3443 - mean_squared_error: 0.3443 - val_loss: 0.3620 - val_mean_squared_error: 0.3579

Epoch 00003: val_mean_squared_error improved from 0.37337 to 0.35792, saving model to 4_2_sim4_combined_9.h5
Epoch 4/50
1000/1000 [==============================] - 329s 329ms/step - loss: 0.3086 - me

/opt/conda/lib/python3.7/site-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


4_2_sim4_combined_9.h5 0.25154651175705856 0.25154651175705856
4_2_sim4_model9
Epoch 1/50
1000/1000 [==============================] - 330s 330ms/step - loss: 0.7724 - mean_squared_error: 0.7724 - val_loss: 0.4244 - val_mean_squared_error: 0.4335

Epoch 00001: val_mean_squared_error improved from inf to 0.43346, saving model to 4_2_sim4_combined_10.h5
Epoch 2/50
1000/1000 [==============================] - 331s 331ms/step - loss: 0.3639 - mean_squared_error: 0.3639 - val_loss: 0.4303 - val_mean_squared_error: 0.4121

Epoch 00002: val_mean_squared_error improved from 0.43346 to 0.41213, saving model to 4_2_sim4_combined_10.h5
Epoch 3/50
1000/1000 [==============================] - 328s 328ms/step - loss: 0.3270 - mean_squared_error: 0.3270 - val_loss: 0.3373 - val_mean_squared_error: 0.3497

Epoch 00003: val_mean_squared_error improved from 0.41213 to 0.34966, saving model to 4_2_sim4_combined_10.h5
Epoch 4/50
 595/1000 [================>.............] - ETA: 2:11 - loss: 0.3108 - mean_

In [11]:
publish_model(1)

/opt/conda/lib/python3.7/site-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


4_2_sim3_combined_1.h5 0.21737819504043962 0.21737819504043962
4_2_sim3_model1
